In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
from time import sleep

In [ ]:
# ecs affidavit portal
url = 'http://103.183.38.66/'
res = requests.get(url)
print(res.status_code)

200


In [17]:
# get 2026 general election ID
general_election_id = 1
res = requests.get(f'http://103.183.38.66/election-settings/get-election-bytype?election_typeID={general_election_id}')
res_dict = res.json()
# print(res_dict)
elections = res_dict['election_schedules']
for election in elections:
    if '2026' in election['date_of_election']:
        print(election['electionID'])
        print(election['election_name'])

478
ত্রয়োদশ জাতীয় সংসদ নির্বাচন ১২-০২-২০২৬


In [5]:
election_id = 478

In [18]:
# get election districts
res = requests.get(f'http://103.183.38.66/election-settings/get-election-zilla?electionID={election_id}')
res_dict = res.json()
# print(res_dict)
zillas = res_dict['zillas']
print(zillas[:5])

[{'zilla_name': 'ঝালকাঠী', 'zillaID': '4'}, {'zilla_name': 'পটুয়াখালী', 'zillaID': '5'}, {'zilla_name': 'পিরোজপুর', 'zillaID': '6'}, {'zilla_name': 'বরগুনা', 'zillaID': '1'}, {'zilla_name': 'বরিশাল', 'zillaID': '2'}]


In [20]:
# get constituency id for each district
all_constituencies = []
for zilla in zillas:
    zillaID = zilla['zillaID']
    print(f'fetching district =', zilla['zilla_name'])
    res = requests.get(f'http://103.183.38.66/election/get-setting-constituency?zillaID={zillaID}&electionID={election_id}')
    res_dict = res.json()
    # print(res_dict)
    constituencies = res_dict['constituencies']
    print(f'constituency count =', len(constituencies))
    for cons in constituencies:
        all_constituencies.append((zillaID, cons['constituencyID'], cons['constituency_name']))
    sleep(2)
print(all_constituencies[:5])

fetching district = ঝালকাঠী
constituency count = 2
fetching district = পটুয়াখালী
constituency count = 4
fetching district = পিরোজপুর
constituency count = 3
fetching district = বরগুনা
constituency count = 2
fetching district = বরিশাল
constituency count = 6
fetching district = ভোলা
constituency count = 4
fetching district = বান্দরবান
constituency count = 1
fetching district = চট্টগ্রাম
constituency count = 16
fetching district = কক্সবাজার
constituency count = 4
fetching district = খাগড়াছড়ি
constituency count = 1
fetching district = রাঙ্গামাটি
constituency count = 1
fetching district = ব্রাহ্মণবাড়িয়া
constituency count = 6
fetching district = চাঁদপুর
constituency count = 5
fetching district = ফেনী
constituency count = 3
fetching district = নোয়াখালী
constituency count = 6
fetching district = লক্ষ্মীপুর
constituency count = 4
fetching district = কুমিল্লা
constituency count = 11
fetching district = মানিকগঞ্জ
constituency count = 3
fetching district = মুন্সীগঞ্জ
constituency count = 3
fetchin

In [21]:
dfcon = pd.DataFrame(all_constituencies, columns=['zillaID', 'constituencyID', 'constituency_name'])
dfcon.head()

,zillaID,constituencyID,constituency_name
0,4,125,ঝালোকাঠি-১
1,4,126,ঝালোকাঠি-২
2,5,111,পটুয়াখালী-১
3,5,112,পটুয়াখালী-২
4,5,113,পটুয়াখালী-৩


In [22]:
dfcon.to_csv('data/constituencies.csv', index=False)

In [3]:
all_constituencies = pd.read_csv('data/constituencies.csv')
all_constituencies.head()

,zillaID,constituencyID,constituency_name
0,4,125,ঝালোকাঠি-১
1,4,126,ঝালোকাঠি-২
2,5,111,পটুয়াখালী-১
3,5,112,পটুয়াখালী-২
4,5,113,পটুয়াখালী-৩


In [9]:
# get candidate info links
candidates = []
for _, cons in all_constituencies.iterrows():
    zillaID = cons['zillaID']
    constituencyID = cons['constituencyID']
    url = f'http://103.183.38.66/get/candidate/data?zilla_id={zillaID}&candidate_type=1&election_id={election_id}&constituency_id={constituencyID}'
    print(f'fetching constituency =', cons['constituency_name'])
    res = requests.get(url)
    soup = bs(res.text, 'html.parser')
    # print(soup.prettify())
    rows = soup.select('tr')
    print(f'candidates =', len(rows))
    for row in rows:
        cols = row.select('td')
        candidate_data = [zillaID, constituencyID]
        for col in cols:
            img = col.select_one('img')
            if img:
                src = img['src']
                candidate_data.append(src)
                continue
            a = col.select_one('a')
            if a:
                href = a['href']
                text = a['download']
                candidate_data.append(f'{text} | {href}')
                continue
            text = col.text.strip()
            if text != '':
                candidate_data.append(text)
                continue
        candidates.append(candidate_data)
    sleep(2)
# print(candidates)

fetching constituency = ঝালোকাঠি-১
candidates = 17
fetching constituency = ঝালোকাঠি-২
candidates = 10
fetching constituency = পটুয়াখালী-১
candidates = 0
fetching constituency = পটুয়াখালী-২
candidates = 0
fetching constituency = পটুয়াখালী-৩
candidates = 0
fetching constituency = পটুয়াখালী-৪
candidates = 0
fetching constituency = পিরোজপুর-১
candidates = 2
fetching constituency = পিরোজপুর-২
candidates = 9
fetching constituency = পিরোজপুর-৩
candidates = 8
fetching constituency = বরগুনা-১
candidates = 6
fetching constituency = বরগুনা-২
candidates = 14
fetching constituency = বরিশাল-১
candidates = 5
fetching constituency = বরিশাল-২
candidates = 10
fetching constituency = বরিশাল-৩
candidates = 10
fetching constituency = বরিশাল-৪
candidates = 5
fetching constituency = বরিশাল-৫
candidates = 10
fetching constituency = বরিশাল-৬
candidates = 6
fetching constituency = ভোলা-১
candidates = 9
fetching constituency = ভোলা-২
candidates = 10
fetching constituency = ভোলা-৩
candidates = 6
fetching constitu

fetching constituency = যশোর-৪
candidates = 10
fetching constituency = যশোর-৫
candidates = 8
fetching constituency = যশোর-৬
candidates = 5
fetching constituency = মাগুড়া-১
candidates = 11
fetching constituency = মাগুড়া-২
candidates = 11
fetching constituency = নড়াইল-১
candidates = 15
fetching constituency = নড়াইল-২
candidates = 9
fetching constituency = বাগেরহাট-১
candidates = 11
fetching constituency = বাগেরহাট-২
candidates = 8
fetching constituency = বাগেরহাট-৩
candidates = 7
fetching constituency = বাগেরহাট-৪
candidates = 6
fetching constituency = খুলনা-১
candidates = 0
fetching constituency = খুলনা-২
candidates = 4
fetching constituency = খুলনা-৩
candidates = 0
fetching constituency = খুলনা-৪
candidates = 0
fetching constituency = খুলনা-৫
candidates = 1
fetching constituency = খুলনা-৬
candidates = 0
fetching constituency = সাতক্ষীরা-১
candidates = 7
fetching constituency = সাতক্ষীরা-২
candidates = 8
fetching constituency = সাতক্ষীরা-৩
candidates = 8
fetching constituency = সাতক্ষীর

In [10]:
dfcand = pd.DataFrame(candidates)
dfcand.head()

,0,1,2,3,4,5,6,7
0,4,125,মোঃ সাব্বির আহমেদ,https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,মোঃ সাব্বির আহমেদ_হলফনামা | https://onss.ecs.g...,মোঃ সাব্বির আহমেদ_নির্বাচনী ব্যয় ও ব্যক্তিগত স...,মোঃ সাব্বির আহমেদ_আয়কর রিটার্ন | https://onss....
1,4,125,মুহাম্মদ মুস্তাফিজুর রহমান,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশ লেবার পার্টি,মুহাম্মদ মুস্তাফিজুর রহমান_হলফনামা | https://o...,মুহাম্মদ মুস্তাফিজুর রহমান_নির্বাচনী ব্যয় ও ব্...,মুহাম্মদ মুস্তাফিজুর রহমান_আয়কর রিটার্ন | http...
2,4,125,মাহমুদা আলম মিতু,https://onss.ecs.gov.bd/document-service/api/v...,জাতীয় নাগরিক পার্টি-এনসিপি,মাহমুদা আলম মিতু_হলফনামা | https://onss.ecs.go...,মাহমুদা আলম মিতু_নির্বাচনী ব্যয় ও ব্যক্তিগত সম...,মাহমুদা আলম মিতু_আয়কর রিটার্ন | https://onss.e...
3,4,125,"এম, মঈন আলম ফিরোজী",https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,"এম, মঈন আলম ফিরোজী_হলফনামা | https://onss.ecs....","এম, মঈন আলম ফিরোজী_নির্বাচনী ব্যয় ও ব্যক্তিগত ...","এম, মঈন আলম ফিরোজী_আয়কর রিটার্ন | https://onss..."
4,4,125,রফিকুল ইসলাম জামাল,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশ জাতীয়তাবাদী দল - বি.এন.পি,রফিকুল ইসলাম জামাল_হলফনামা | https://onss.ecs....,রফিকুল ইসলাম জামাল_নির্বাচনী ব্যয় ও ব্যক্তিগত ...,রফিকুল ইসলাম জামাল_আয়কর রিটার্ন | https://onss...


In [11]:
dfcand.shape

(2447, 8)

In [12]:
dfcand.tail()

,0,1,2,3,4,5,6,7
2442,32,67,মোঃ আসাদুল হক,https://onss.ecs.gov.bd/document-service/api/v...,আমজনতার দল,মোঃ আসাদুল হক_হলফনামা | https://onss.ecs.gov....,মোঃ আসাদুল হক_নির্বাচনী ব্যয় ও ব্যক্তিগত সম্প...,মোঃ আসাদুল হক_আয়কর রিটার্ন | https://onss.ecs...
2443,32,67,"এস, এম, সাইফ মোস্তাফিজ",https://onss.ecs.gov.bd/document-service/api/v...,জাতীয় নাগরিক পার্টি-এনসিপি,"এস, এম, সাইফ মোস্তাফিজ_হলফনামা | https://onss....","এস, এম, সাইফ মোস্তাফিজ_নির্বাচনী ব্যয় ও ব্যক্ত...","এস, এম, সাইফ মোস্তাফিজ_আয়কর রিটার্ন | https://..."
2444,32,67,ইলোরা খাতুন,https://onss.ecs.gov.bd/document-service/api/v...,জাতীয় সমাজতান্ত্রিক দল-জেএসডি,ইলোরা খাতুন_হলফনামা | https://onss.ecs.gov.bd/...,ইলোরা খাতুন_নির্বাচনী ব্যয় ও ব্যক্তিগত সম্পদের...,ইলোরা খাতুন_আয়কর রিটার্ন | https://onss.ecs.go...
2445,32,67,মোঃ আনোয়ার হোসেন,https://onss.ecs.gov.bd/document-service/api/v...,বাংলাদেশের সমাজতান্ত্রিক দল-বাসদ,মোঃ আনোয়ার হোসেন_হলফনামা | https://onss.ecs...,মোঃ আনোয়ার হোসেন_নির্বাচনী ব্যয় ও ব্যক্তিগত...,মোঃ আনোয়ার হোসেন_আয়কর রিটার্ন | https://ons...
2446,32,67,মোঃ হুমায়ন কবির,https://onss.ecs.gov.bd/document-service/api/v...,স্বতন্ত্র,মোঃ হুমায়ন কবির_হলফনামা | https://onss.ecs.go...,মোঃ হুমায়ন কবির_নির্বাচনী ব্যয় ও ব্যক্তিগত সম...,মোঃ হুমায়ন কবির_আয়কর রিটার্ন | https://onss.e...


In [14]:
dfcand.columns = ['zillaID', 'constituencyID', 'name', 'img', 'party', 'affidavit', 'election_fund', 'tax_return']

In [15]:
dfcand.to_csv('data/candidates.csv', index=False)